In [18]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

from gensim.models import Phrases

import eli5

pd.options.mode.chained_assignment = None

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sanya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Считывание датасета

In [19]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv",  encoding = 'utf8', sep = '\t')
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [20]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

## Предобработка

In [21]:
df = df[df['sentiment'] != 'neautral']
df.iloc[0]['review']

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [22]:
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))

df['review_processed'].iloc[0]

100%|██████████| 60000/60000 [00:04<00:00, 14399.14it/s]


['качество',
 'плохое',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'Фото',
 'не',
 'соответствует',
 'Ткань',
 'ужасная',
 'рисунок',
 'блеклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'УЖАС',
 'не',
 'стоит',
 'за',
 'такие',
 'деньги',
 'г']

## Лемматизация

In [23]:
morph = pymorphy2.MorphAnalyzer()
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 60000/60000 [01:45<00:00, 571.09it/s]


In [24]:
morph.parse('сделали')[0].normal_form

'сделать'

## Преобразование TF-IDF

In [25]:
vectorizer = TfidfVectorizer(ngram_range = (1, 2), lowercase = True) 
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))

X.shape

(60000, 396100)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size = 0.3)

## Обучение модели

In [27]:
logit = LogisticRegression()
logit.fit(X_train, y_train)

y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.9782914095609576

## Отображение весов слов

In [28]:
eli5.show_weights(estimator = logit, 
                  feature_names= list(vectorizer.get_feature_names()),
                  top = (20, 20))

c:\Users\Sanya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+10.526,отличный
+9.873,хороший
+8.079,супер
+7.358,спасибо
+7.071,немного
+6.744,хорошо
+6.390,отлично
+6.284,классный
+6.151,приятный
+5.994,довольный


## Обучение модели на биграммах

In [35]:
bigram = Phrases(df['review_lemmatized'].values, min_count = 2, threshold = 7)
df['review_lemmatized_coll'] = df['review_lemmatized'].progress_apply(lambda x: bigram[x])

100%|██████████| 60000/60000 [00:01<00:00, 43010.77it/s]


In [36]:
vectorizer = TfidfVectorizer(ngram_range = (1, 1))
X = vectorizer.fit_transform(df['review_lemmatized_coll'].apply(lambda x: ' '.join(x)))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size = 0.3)

In [38]:
logit = LogisticRegression()
logit.fit(X_train, y_train)

y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.972119433214052

## Обучение с использованием GridSearch

In [41]:
parametrs = {'C': [0.5, 1, 10, 20, 100],
             'max_iter': [50, 100, 300, 500]}

In [ ]:
clf = GridSearchCV(logit, parametrs)
clf.fit(X_train, y_train)

In [43]:
clf.best_params_

{'C': 10, 'max_iter': 50}

In [45]:
logit = LogisticRegression(C = 10, max_iter = 50)
logit.fit(X_train, y_train)

y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

c:\Users\Sanya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9748080063762106